# Домашнее задание 4. Конструирование текстовых признаков из твитов пользователей

## Сбор данных

Сначала импортируем все чем будем пользоваться

In [3]:
from __future__ import print_function
from sys import stdout
from time import sleep
from collections import defaultdict
import codecs
import pickle

import twitter
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from nltk.stem.wordnet import WordNetLemmatizer
from pytagcloud import create_tag_image, make_tags
import nltk

Первый этап - сбор твитов пользователей. Необходимо подключаться к Twitter API и запрашивать твиты по id пользователя. 
Подключение к API подробно описано в ДЗ 1.

In [4]:

CONSUMER_KEY = "Il7TATLVOo64Fvdzz1XPBscBD"
CONSUMER_SECRET = "EaZBgG2QAGjbb3nASxzbXUwBhkmj0P1geXP8EtiuNxAm484rsD"

ACCESS_TOKEN_KEY = "3064704131-3zaVpY40gSd1kAP163yoMAlP2tqcBLG11qYjIh8"
ACCESS_TOKEN_SECRET = "cnIzGKo5NQPwTVQegIqT5Lv7IeeLCvznNn3lgouzIEOrK"

api = twitter.Api(consumer_key=CONSUMER_KEY, 
                  consumer_secret=CONSUMER_SECRET, 
                  access_token_key=ACCESS_TOKEN_KEY, 
                  access_token_secret=ACCESS_TOKEN_SECRET)

Для получения твитов пользователя может быть использован метод GetUserTimeline из библиотеки python-twitter. Он позволяет получить не более 200 твитов пользователя.

Метод имеет ограничение по количеству запросов в секунду. Для получения информации о промежутке времени, которое необходимо подождать для повторного обращения к API может быть использован метод `GetSleepTime`. Для получения информации об ограничениях запросов с помощью метода `GetUserTimeLine` необходимо вызывать `GetSleepTime` с параметром "statuses/user_timeline".

Метод GetUserTimeline возвращает объекты типа Status. У этих объектов есть метод AsDict, который позволяет представить твит в виде словаря.

Id пользователей необходимо считать из файла, как было сделано в ДЗ 1.

Необходимо реализовать функцию `get_user_tweets(user_id)`. Входной параметр - id пользователя из файла. Возвращаемое значение - массив твитов пользователя, где каждый твит представлен в виде словаря. Предполагается, что информация о пользователе содержится в твитах, которые пользователь написал сам. Это означает, что можно попробовать отфильтровать ответы другим пользователям, ссылки и ретвиты, а так же картинки и видео, так как наша цель - найти текстовую информацию.

In [5]:
def get_user_tweets(user_id):
    return [st.AsDict() for st in api.GetUserTimeline(user_id=user_id)]

В качестве теста получим последние твиты кого-то кто назвался Bred Pit :)

In [6]:
pit_tweets = get_user_tweets(144808131)
pit_texts = [dt['text'] for dt in pit_tweets]

print(pit_texts)

[u'RT @sashasasha3506: http://t.co/VNhzKyiKA0', u'RT @AknKarc: "Don\'t act, be yourself !"', u'WTF= Welcome To Facebook :)', u'gotta be kidding...umm', u'so far away...', u'soo good']


## Разбор текста твита

Обработка текста предполагает разбиение текста на отдельные элементы - параграфы, предложения, слова. Мы будем преобразовывать текст твита к словам. Для этого текст необходимо разбить на слова. Сделать это можно, например, с помощью регулярного выражения.

Необходимо реализовать функцию, `get_words(text)`. Входной параметр - строка с текстом. Возвращаемое значение - массив строк (слов). Обратите внимание, что нужно учесть возможное наличие пунктуации и выделить по возможности только слова. 

In [7]:
def get_words(text):
    """returns list of words"""
    return nltk.word_tokenize(text)

In [8]:
print(get_words("Here are different words!"))

['Here', 'are', 'different', 'words', '!']


Далее полученные слова необходимо привести к нормальной форме. То есть привести их к форме единственного числа настоящего времени и пр. Сделать это можно с помощью библиотеки nltk. Информацию по загрузке, установке библиотеки и примерах использования можно найти на сайте http://www.nltk.org/

Для загрузки всех необходимых словарей можно воспользоваться методом download из библиотеки nltk.

Но так как мы все загрузили на занятии, смело уберем его.

Для дальнейшей обработки слова должны быть приведены к нижнему регистру. 

Для приведения к нормальной форме можно использовать `WordNetLemmatizer` из библиотеки nltk. У этого класса есть метод `lemmatize`.

Также необходимо убрать из текста так называемые стоп-слова. Это часто используемые слова, не несущие смысловой нагрузки для наших задач. Сделать это можно с помощью `stopwords` из nltk.corpus

Необходимо реализовать функцию `get_tokens(words)`. Входной параметр - массив слов. Возвращаемое значение - массив токенов.

In [9]:
def get_tokens(words):
    stop = nltk.corpus.stopwords.words('english')
    wnl = WordNetLemmatizer()
    words = [wnl.lemmatize(wrd.lower()) for wrd in words]
    return[wrd for wrd in words if wrd not in stop]


In [10]:
print(get_tokens(["here", "are", "different", "words"]))

['different', u'word']


Необходимо реализовать функцию `get_tweet_tokens(tweet)`. Входной параметр - текст твита. Возвращаемое значение -- токены твита. 

Здесь наивным образом офильтруем фото, видео и ссылки

In [11]:
def get_tweet_tokens(tweet):
    if "http:" not in tweet and "/" not in tweet and "@" not in tweet:
        return get_tokens(get_words(tweet))
    else:
        return list()

Необходимо реализовать функцию `collect_users_tokens()`. Функция должна сконструировать матрицу признаков пользователей. В этой матрице строка - пользователь. Столбец - токен. На пересечении - сколько раз токен встречается у пользователя.
Для построения матрицы можно использовать `DictVectorizer` из `sklearn.feature_extraction`.

In [12]:
def collect_users_tokens(src_tweets):
    """returns users list and list of user dicts. Each dict contains frequence of user tokens"""
    id_to_tokens = dict()
    for dt in src_tweets:
        if dt["user"]["id"] not in id_to_tokens.keys():
            id_to_tokens[dt["user"]["id"]] = dict()
        
        text = dt["text"]
        tokens = get_tweet_tokens(text)
        
        for tk in tokens:
            if tk not in id_to_tokens[dt["user"]["id"]].keys():
                id_to_tokens[dt["user"]["id"]][tk] = 0
            id_to_tokens[dt["user"]["id"]][tk] += 1
    
    ids = list(id_to_tokens.keys())
    
    return ids, [id_to_tokens[i] for i in ids]


In [13]:
TRAINING_SET_URL = "../files/twitter_train.txt"
df_users = pd.read_csv(TRAINING_SET_URL, sep=",", header=0)

src_tweets = list()

print ("Total users %s:" % len(df_users["Id"]))

for el, id_ in enumerate(df_users["Id"][:40]):
    if el % 100 == 0:
        print("Processed %s users" % el)
        stdout.flush()
    try:
        src_tweets += get_user_tweets(id_)
    except:
        tm = api.GetSleepTime("statuses/user_timeline")
        print("Sleeping %s" % tm)
        stdout.flush()
        sleep(tm)

with open("../files/src_tweets.dat", "w") as f:
    pickle.dump(src_tweets, f)


Total users 8000:
Processed 0 users
Sleeping 0
Sleeping 0


In [14]:
users, users_tokens = collect_users_tokens(src_tweets)
v = DictVectorizer()
vs = v.fit_transform(users_tokens)

Давайте чуть-чуть посмотрим на получившиеся наборы слов.

In [15]:
print(users[:10], users_tokens[:10])

[984121344, 601849857, 215056389, 248807431, 61931528, 2352595372, 102137866, 391413781, 2962522134, 1683980311] [{u'wid': 1, u'hell': 1, u'resurrected': 1, u'feel': 2, u'mom': 1, u'fuck': 1, u'dead': 1, u'one': 1, u'today': 2, u'done': 1, u'ur': 1, u'year': 1, u'watched': 1, u'still': 1, u'lolita': 2, u'technically': 1, u'hydrocodone': 1, u'thigh': 1, u'yoga': 1, u'stair': 1, u'floor': 1, u'wtfff': 1, u'movie': 1, u'torn': 1, u'actually': 1, u'book': 1, u'loll': 1, u'pissed': 1, u'sick': 1, u'2013': 1, u'gave': 1, u'gravestone': 1, u'head': 1, u'full': 1, u"'s": 1, u'boy': 1, u'read': 1, u'class': 1, u'burn': 1, u'\U0001f47d\U0001f60e': 1, u'broke': 1, u'fucked': 1, u"'m": 2, u'cara': 1, u'totally': 1, u'mcr': 1, u'died': 1, u'going': 1, u'like': 2, u'getting': 1, u'leather': 1, u'die': 1, u'favorite': 1, u'fux': 1, u"'ve": 1, u'jacket': 1, u'length': 1, u'lingerie': 1, u'nose': 1, u'aesthetic': 1, u'history': 1}, {}, {u'swaggy': 1, u'emotion': 1, u'right': 1, u'love': 3, u'illegally'

Сохраним полученные данные в файл. Используется метод savez из numpy. 

In [16]:
np.savez("../files/out_4.dat", data=vs, users=users, users_tokens=users_tokens )

Далее для получения представления о полученной информацию о токенах предлагается отобразить ее в виде облака тэгов. [Подсказка](http://anokhin.github.io/img/tag_cloud.png). 

In [17]:
def draw_tag_cloud(v, vs):
    """Draws tag cloud of found tokens"""
    res = dict()
    for dt in vs:
        for el in dt:
            try:
                el1 = el.encode('ascii')
                if el1 not in res.keys():
                    res[el1] = 0
                res[el1] += 1
            except:
                pass
    tags = make_tags([(k, res[k]) for k in res])
    create_tag_image(tags[:100], '/Users/feldsherov/Desktop/cloud_large.png', fontname='Molengo')

In [18]:
draw_tag_cloud(v, users_tokens)